In [1]:
#This the demodulation part of this code was submitted as an assignment for CS4020 course in 2023 in University of South Eastern Norway (USN)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, sosfilt, hilbert
from scipy.fft import fft, fftfreq



#  band pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def bandpass_signal(signal, fs, lowcut, highcut):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(2, [low, high], btype='band', analog=False, output='sos')
    y = sosfilt(sos, signal)
    return y

#  low pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def low_signal(signal, lowcut, fs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    order = 2
    sos1 = butter(order, low, btype='low', output='sos')
    l = sosfilt(sos1, signal)
    return l

#  low pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def calculate_spectrum(data, fs, lowcut, highcut):
    filtered_signal = bandpass_signal(data, fs, lowcut, highcut)
    analytic_signal = hilbert(filtered_signal)
    envelope = np.abs(analytic_signal)
    lowpass_signal = low_signal(envelope, lowcut, fs)
    
    fft_result = fft(lowpass_signal)
    freqs = fftfreq(len(lowpass_signal), 1/fs)[:len(lowpass_signal)//2]
    amplitude = 2.0 / len(lowpass_signal) * np.abs(fft_result[0:len(lowpass_signal)//2])
    return freqs, amplitude

def plot_frequency_spectrum(freqs, amplitude, output_path):
    fig, ax = plt.subplots()
    ax.plot(freqs, amplitude)
    ax.set_title("Frequency Spectrum")
    ax.set_xlabel("Frequency (Hz)")
    ax.set_ylabel("Amplitude")
    plt.savefig(output_path, dpi=300)
    plt.close(fig)

def newdataset(path, date, location, first, last):
    fs = 20000
    lowcut = 2000
    highcut = 9900
    
    file_path = os.path.join(path, date)

    # Load the vibration data from the file
    dataset = pd.read_csv(file_path, sep='\t', header=None)
    data = dataset.iloc[:, 4].values

    freqs, amplitude = calculate_spectrum(data, fs, lowcut, highcut)

    directory_path = r"C:\Users\zahra\nasa\filter\wavelet13"  
    file_name = f"{date}_frequency_spectrum.png"
    output_path = os.path.join(directory_path, file_name)
    

path = "C:/Users/zahra/Thesis/datasets/1st_test"
dates = os.listdir(path)
for date in dates[-47:]: #based on the previous detection step
    location = dates.index(date)
    newdataset(path, date, location, dates[0], dates[-1])
